# Mức 3: Đánh giá mô hình deraining (đã fine-tune) trên dữ liệu ngoài bài báo
**CHÚ Ý:**
- Bật GPU T4 x2 trước khi chạy notebook
- Thêm các Kaggle dataset sau làm input:
    - For pre-trained weight: https://kaggle.com/datasets/48e9d5bc84d59d1aaee28d770c6ceae05bcbc151efe901d644ab2a354afae48b
    - For test set: https://kaggle.com/datasets/3e283f0772a1cb7e19dbea29780e1ad5459415b323b7e3a4b074a1e0263f4898

# Môi trường & Dependencies
- Kaggle

In [29]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.3 LTS"
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [30]:
# !rm -rf /kaggle/working/*

In [31]:
# !sudo apt-get install libtinfo5

In [32]:
!python --version
!python3.8 --version

Python 3.10.12
/bin/bash: line 1: python3.8: command not found


In [33]:
# !sudo apt update
# !sudo apt install python3.8 python3.8-distutils
# !sudo apt install python3.8-dev

In [34]:
# !python --version
# !python3.8 --version

In [35]:
# %cd /kaggle
# !wget https://bootstrap.pypa.io/get-pip.py
# !sudo python3.8 get-pip.py

In [36]:
!pip --version
!pip3.8 --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
/bin/bash: line 1: pip3.8: command not found


In [37]:
# !python3.8 -m pip install torch==1.8.1 torchvision==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

In [4]:
!pip show torch torchvision

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, pytorch-msssim, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision
---
Name: torchvision
Version: 0.20.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, pillow, torch
Required-by: easyocr, fastai, timm


In [2]:
# !pip3.8 install tensorboard einops scikit-image pytorch_msssim opencv-python
!pip install tensorboard einops scikit-image pytorch_msssim opencv-python

In [3]:
!pip show tensorboard einops scikit-image pytorch_msssim opencv-python

Name: tensorboard
Version: 2.17.1
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, grpcio, markdown, numpy, packaging, protobuf, setuptools, six, tensorboard-data-server, werkzeug
Required-by: tensorflow
---
Name: einops
Version: 0.8.0
Summary: A new flavour of deep learning operations
Home-page: https://github.com/arogozhnikov/einops
Author: Alex Rogozhnikov
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 
---
Name: scikit-image
Version: 0.25.0
Summary: Image processing in Python
Home-page: https://scikit-image.org
Author: 
Author-email: 
License: Files: *
         Copyright: 2009-2022 the scikit-image team
         License: BSD-3-Clause
         
         Files: doc/source/themes/scikit-image/layout.html
         Copyright: 2007-2010 

# Copy source code

In [39]:
!cp -r "/kaggle/input/sfnet-source-code-and-model/SFNet" \
    "/kaggle/working/"

In [40]:
!cp "/kaggle/input/sfnet-source-code-and-model/test_derain_muc_2.py" \
    "/kaggle/working/SFNet/Image_deraining/"

In [41]:
!ls /kaggle/working

results  SFNet	test_data


# Kiểm tra dữ liệu đã thêm vào Kaggle Input

In [42]:
!ls /kaggle/input/sfnet-source-code-and-model

deraining.pkl		  fixed_main_for_deraining.py  test_derain_muc_2b.py
deraining_runtime_weight  SFNet			       test_derain_muc_2.py


In [43]:
!ls /kaggle/input/raindrop-data

raindrop_data


# Copy dữ liệu từ Kaggle Input

In [44]:
!mkdir -p /kaggle/working/test_data/test_a/input/
!mkdir -p /kaggle/working/test_data/test_a/target/
!mkdir -p /kaggle/working/test_data/test_b/input/
!mkdir -p /kaggle/working/test_data/test_b/target/

In [45]:
!cp "/kaggle/input/raindrop-data/raindrop_data/test_a/test_a/data/"* \
    "/kaggle/working/test_data/test_a/input/"

!cp "/kaggle/input/raindrop-data/raindrop_data/test_a/test_a/gt/"* \
    "/kaggle/working/test_data/test_a/target/"

!cp "/kaggle/input/raindrop-data/raindrop_data/test_b/test_b/data/"* \
    "/kaggle/working/test_data/test_b/input/"

!cp "/kaggle/input/raindrop-data/raindrop_data/test_b/test_b/gt/"* \
    "/kaggle/working/test_data/test_b/target/"

In [46]:
import os

folder_paths = [
    "/kaggle/working/test_data/test_a/input/",
    "/kaggle/working/test_data/test_a/target/",
    "/kaggle/working/test_data/test_b/input/",
    "/kaggle/working/test_data/test_b/target/",
]

for folder_path in folder_paths:
    for filename in os.listdir(folder_path):
        ext = "png" if "test_a" in folder_path else "jpg"
        postfix = "_rain" if "input" in folder_path else "_clean"
        
        if filename.endswith(f"{postfix}.{ext}"):
            new_name = filename.replace(f"{postfix}.{ext}", f".{ext}")
            old_path = os.path.join(folder_path, filename)
            new_path = os.path.join(folder_path, new_name)
            os.rename(old_path, new_path)
            print(f'Renamed: {old_path} -> {new_path}')

Renamed: /kaggle/working/test_data/test_a/input/31_rain.png -> /kaggle/working/test_data/test_a/input/31.png
Renamed: /kaggle/working/test_data/test_a/input/15_rain.png -> /kaggle/working/test_data/test_a/input/15.png
Renamed: /kaggle/working/test_data/test_a/input/47_rain.png -> /kaggle/working/test_data/test_a/input/47.png
Renamed: /kaggle/working/test_data/test_a/input/41_rain.png -> /kaggle/working/test_data/test_a/input/41.png
Renamed: /kaggle/working/test_data/test_a/input/9_rain.png -> /kaggle/working/test_data/test_a/input/9.png
Renamed: /kaggle/working/test_data/test_a/input/6_rain.png -> /kaggle/working/test_data/test_a/input/6.png
Renamed: /kaggle/working/test_data/test_a/input/22_rain.png -> /kaggle/working/test_data/test_a/input/22.png
Renamed: /kaggle/working/test_data/test_a/input/39_rain.png -> /kaggle/working/test_data/test_a/input/39.png
Renamed: /kaggle/working/test_data/test_a/input/4_rain.png -> /kaggle/working/test_data/test_a/input/4.png
Renamed: /kaggle/working/

# Inference

In [47]:
# # %cd /content/drive/MyDrive/advanced_machine_learning/colab/SFNet/Motion_deblurring
# %cd /kaggle/SFNet
# %cd pytorch-gradual-warmup-lr/
# # !python3.8 setup.py install
# !python setup.py install
# # !python3.10 setup.py install
# %cd ..

In [48]:
# !cp /kaggle/SFNet/Image_dehazing/eval.py /kaggle/SFNet/Image_deraining
# !cp /kaggle/SFNet/Motion_deblurring/eval.py /kaggle/SFNet/Image_deraining

In [49]:
!mkdir /kaggle/working/SFNet/Image_deraining/results

In [50]:
%cd /kaggle/working/SFNet/Image_deraining
%pwd
%ls

/kaggle/working/SFNet/Image_deraining
data/             models/       results/              train.py
deraining_test.m  __pycache__/  test_derain_muc_2.py  utils.py
main.py           README.md     test.py               valid.py


In [51]:
# !python test.py
# !python test_derain_muc_2.py --test_model /kaggle/input/sfnet-source-code-and-model/deraining.pkl --data_dir /kaggle/working/test_data/ # --model_save_dir /kaggle/working/output

In [52]:
!python test_derain_muc_2.py --test_model /kaggle/input/sfnet-source-code-and-model/deraining_runtime_weight/model_420.pkl \
    --data_dir /kaggle/working/test_data/ # --model_save_dir /kaggle/working/output

/kaggle/working/SFNet/Image_deraining/test_derain_muc_2.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(args.test_model)
results/SFNet/deraining

# Download dữ liệu
- Kết quả đã được upload tại link: https://studenthcmusedu-my.sharepoint.com/:f:/g/personal/24c11071_student_hcmus_edu_vn/ElXHDgJETABLo6DGBz430k0BLUT4woN30WnWlj60RwoZhQ?e=kjzb0b

In [53]:
!mv /kaggle/working/SFNet/Image_deraining/results /kaggle/working/results

In [54]:
%cd /kaggle/working/results

/kaggle/working/results


In [55]:
# Zip kết quả inference
!zip -r deraining_test_output.zip .

updating: SFNet/ (stored 0%)
updating: SFNet/deraining/ (stored 0%)
updating: SFNet/deraining/test_a/ (stored 0%)
updating: SFNet/deraining/test_a/45.png (deflated 0%)
updating: SFNet/deraining/test_a/57.png (deflated 0%)
updating: SFNet/deraining/test_a/28.png (deflated 0%)
updating: SFNet/deraining/test_a/55.png (deflated 0%)
updating: SFNet/deraining/test_a/35.png (deflated 0%)
updating: SFNet/deraining/test_a/29.png (deflated 0%)
updating: SFNet/deraining/test_a/40.png (deflated 0%)
updating: SFNet/deraining/test_a/8.png (deflated 0%)
updating: SFNet/deraining/test_a/3.png (deflated 0%)
updating: SFNet/deraining/test_a/30.png (deflated 0%)
updating: SFNet/deraining/test_a/17.png (deflated 0%)
updating: SFNet/deraining/test_a/7.png (deflated 0%)
updating: SFNet/deraining/test_a/43.png (deflated 0%)
updating: SFNet/deraining/test_a/5.png (deflated 0%)
updating: SFNet/deraining/test_a/12.png (deflated 0%)
updating: SFNet/deraining/test_a/25.png (deflated 0%)
updating: SFNet/deraining/

In [56]:
%cd /kaggle/working/results
!ls

/kaggle/working/results
deraining_test_output.zip  results  SFNet


# What's next?
- **Chỉ xét thư mục test_b, vì thư mục test_a đã được sử dụng làm valid data**
- Kết quả inference được upload tại: https://studenthcmusedu-my.sharepoint.com/:u:/g/personal/24c11071_student_hcmus_edu_vn/Ef9S8rDyS_VJg0wbvjQp-fIBW6ZtR1szYd_RTlwNo3M8MQ?e=9cZ6vs
- Ảnh tạo ra cần resize về kích thước của ground truth tương ứng trước khi evaluate. Kết quả resize được upload tại: 
- Sau khi download dữ liệu được inference &rarr; Sử dụng code Matlab ([SFNet/Image_deraining/deraining_test.m](../../../docs/original_source_code/SFNet/Image_deraining/deraining_test.m)) để evaluate
- Ghi chú cho Matlab:
    - Có thể dùng online Matlab trial 
    - Bỏ các dòng có lời gọi hàm gcp nếu tài khoản Matlab không có license cho [Parallel Computing Toolbox](https://www.mathworks.com/products/parallel-computing.html) - a tool for performing parallel computations on multicore computers, GPUs, and clusters